In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [0]:
tf.disable_eager_execution()

In [3]:
import numpy as np
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Defining methods to classify the images with labels
WIDTH = 128
HEIGHT = 128
def classes_to_int(label):
    # label = classes.index(dir)
    label = label.strip()
    if label == "Black-grass":  return 0
    if label == "Charlock":  return 1
    if label == "Cleavers":  return 2
    if label == "Common Chickweed":  return 3
    if label == "Common wheat":  return 4
    if label == "Fat Hen":  return 5
    if label == "Loose Silky-bent": return 6
    if label == "Maize":  return 7
    if label == "Scentless Mayweed": return 8
    if label == "Shepherds Purse": return 9
    if label == "Small-flowered Cranesbill": return 10
    if label == "Sugar beet": return 11
    print("Invalid Label", label)
    return 12

In [0]:
# Create method to read data from the Traning dataset
def readTrainData(trainDir):
    data = []
    labels = []
    # loop over the input images
    dirs = os.listdir(trainDir) 
    print('Dirs::', dirs)
    for dir in dirs:
        absDirPath = os.path.join(os.path.sep,trainDir, dir)
        images = os.listdir(absDirPath)
        xC=0
        for imageFileName in images:
            xC=xC+1
            # load the image, pre-process it, and store it in the data list
            imageFullPath = os.path.join(trainDir, dir, imageFileName)
            #print(imageFullPath)
            img = load_img(imageFullPath)
            arr = img_to_array(img)  # Numpy array with shape (128,128,3)
            arr = cv2.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
            print(arr.shape, " -- " , imageFullPath) 
            data.append(arr)
            label = classes_to_int(dir)
            labels.append(label)
            if xC == 50:
              print("50 Files read noe breaking from loop")
              break;

    return data, labels

Creating train and validation set(test)

In [7]:
X, Y = readTrainData("/content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/")
# scale the raw pixel intensities to the range [0, 1]
X = np.array(X, dtype="float") / 255.0
Y = np.array(Y)
# convert the labels from integers to vectors
Y =  to_categorical(Y, num_classes=12)

print("Data split with 80 and 20")
(trainX, valX, trainY, valY) = train_test_split(X,Y,test_size=0.2, random_state=42)

Dirs:: ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet', 'Loose Silky-bent', 'Maize']
(128, 128, 3)  --  /content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/Black-grass/0260cffa8.png
(128, 128, 3)  --  /content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/Black-grass/078eae073.png
(128, 128, 3)  --  /content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/Black-grass/0ace21089.png
(128, 128, 3)  --  /content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/Black-grass/0bb75ded8.png
(128, 128, 3)  --  /content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/Black-grass/0c67c3fc3.png
(128, 128, 3)  --  /content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/train/Black-grass/0d4f

In [8]:
print(len(trainX[0]))
print(len(trainY))
print('##')
print(len(valX))
print(len(valY))

128
480
##
120
120


Defining Initial Model

In [0]:
def myModel():
    model = Sequential()
    # first set of CONV => RELU => POOL layers
    # The CONV  layer will learn 20 convolution filters, each of which are 5×5.
    model.add(Conv2D(32, (3,3), padding="same", input_shape=(128,128,3)))
    #model.add(BatchNormalization()) 
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(64, (3,3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(128, (3,3), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=500))
    model.add(Activation("relu"))
    model.add(Dense(units=12))
    model.add(Activation("softmax"))
    opt = Adam(lr=-0.2, decay=-0.2 / 42)
    # use binary_crossentropy if there are two classes
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model

In [0]:
model = KerasClassifier(build_fn=myModel, verbose=0)

In [0]:
# Setting Grid Search parameters
batch_size = [30];
epochs = [15];
param_grid = dict(batch_size=batch_size, epochs=epochs);
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1,cv=3);
grid_result = grid.fit(trainX, trainY);

In [0]:
print(grid)

In [0]:
print(grid_result.best_params_)

In [0]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params'] 
for mean, stdev, param in zip(means, stds, params): 
  print("%f (%f) with: %r" % (mean, stdev, param))

In [0]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")
model = myModel()

In [14]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
activation_12 (Activation)   (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 128)      

In [15]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=32), \
    validation_data=(valX, valY), \
    steps_per_epoch=len(trainX) // 32, epochs=20, verbose=1)

Epoch 1/20
15/15 [==============================] - 23s 2s/step - loss: 13.8331 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 2/20
15/15 [==============================] - 21s 1s/step - loss: 14.7078 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 3/20
15/15 [==============================] - 21s 1s/step - loss: 14.7078 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 4/20
15/15 [==============================] - 21s 1s/step - loss: 14.7078 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 5/20
15/15 [==============================] - 21s 1s/step - loss: 14.7078 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 6/20
15/15 [==============================] - 21s 1s/step - loss: 14.7078 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 7/20
15/15 [==============================] - 21s 1s/step - loss: 14.7078 - acc: 0.0875 - val_loss: 15.0436 - val_acc: 0.0667
Epoch 8/20
15/15 [==============================] - 21s 1s/step - loss: 14.7

In [16]:
H.history

{'acc': [0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875,
  0.0875],
 'loss': [13.833077669143677,
  14.70776195526123,
  14.70776170094808,
  14.707761764526367,
  14.707761891682942,
  14.707761828104655,
  14.707761764526367,
  14.707761828104655,
  14.707761764526367,
  14.707762018839519,
  14.707761828104655,
  14.707761764526367,
  14.70776170094808,
  14.707761764526367,
  14.707761891682942,
  14.707761828104655,
  14.707761828104655,
  14.707761764526367,
  14.70776170094808,
  14.707761828104655],
 'val_acc': [0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.06666666691501935,
  0.0666666669

In [0]:
def readTestData(testDir):
    data = []
    filenames = []
    # loop over the input images
    images = os.listdir(testDir)
    xT=0;
    for imageFileName in images:
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join(testDir, imageFileName)
        #print(imageFullPath)
        img = load_img(imageFullPath)
        arr = img_to_array(img)  # Numpy array with shape (...,..,3)
        arr = cv2.resize(arr, (HEIGHT,WIDTH)) 
        data.append(arr)
        filenames.append(imageFileName)
        #print(imageFileName)
        xT=xT+1;
        if xT == 100:
          print("Testing for first 100 files only")
          break;
    return data, filenames

In [25]:
testX, filenames = readTestData("/content/drive/My Drive/Colab Notebooks/R7/Project1/plant-seedlings-classification/test/")
# scale the raw pixel intensities to the range [0, 1]
testX = np.array(testX, dtype="float") / 255.0

yFit = model.predict(testX, batch_size=10, verbose=1)
print(yFit) 
print(type(yFit)) # numpy.ndarray
print(type(filenames)) # list


Testing for first 100 files only
100/100 [==============================] - 1s 11ms/step
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
<class 'list'>


In [0]:
def myOptimizedModel():
    model2 = Sequential()
    # first set of CONV => RELU => POOL layers
    # The CONV  layer will learn 20 convolution filters, each of which are 5×5.
    model2.add(Conv2D(32, (3,3), padding="same", input_shape=(128,128,3)))
    model2.add(BatchNormalization()) 
    model2.add(Activation("relu"))
    model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model2.add(Conv2D(64, (3,3), padding="same"))
    model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model2.add(Conv2D(128, (3,3), padding="same"))
    model2.add(BatchNormalization())
    model2.add(Activation("relu"))
    model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model2.add(Flatten())
    model2.add(Dense(units=500))
    model2.add(Activation("relu"))
    model2.add(Dense(units=300))
    model2.add(Activation("relu"))
    model2.add(Dense(units=12))
    model2.add(Activation("softmax"))
    opt = Adam(lr=-0.28, decay=-0.28 / 42)
    # use binary_crossentropy if there are two classes
    model2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model2

In [0]:
from keras.layers import BatchNormalization

In [30]:
modelOpt = myOptimizedModel()
H2 = modelOpt.fit_generator(aug.flow(trainX, trainY, batch_size=41), \
    validation_data=(valX, valY), \
    steps_per_epoch=len(trainX) // 41, epochs=1, verbose=1)

Epoch 1/1
11/11 [==============================] - 29s 3s/step - loss: 13.6610 - acc: 0.0887 - val_loss: 14.9092 - val_acc: 0.0750
